## This notebook is for evaluating the performance of each server from the 3 servers ##
1. MySQL
2. MSSQL (Microsoft SQL Server)
3. PostgreSQL

### Importing the required libraries ###

In [1]:
import time, random

from dotenv import load_dotenv
import os

import pandas as pd, matplotlib.pyplot as plt, seaborn as sns
%matplotlib inline
sns.set()

### Setting the dataframe that will contain the results ###

In [2]:
servers_performance = pd.DataFrame(columns=["MySQL/INSERT", "MySQL/UPDATE", "MSSQL/INSERT", "MSSQL/UPDATE", "PostgreSQL/INSERT", "PostgreSQL/UPDATE"],
                                   index=range(1, 201))
servers_performance = servers_performance.astype(float)
servers_performance.index.name = "StatementID"

In [3]:
servers_performance

,MySQL/INSERT,MySQL/UPDATE,MSSQL/INSERT,MSSQL/UPDATE,PostgreSQL/INSERT,PostgreSQL/UPDATE
StatementID,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
196,NaN,NaN,NaN,NaN,NaN,NaN
197,NaN,NaN,NaN,NaN,NaN,NaN
198,NaN,NaN,NaN,NaN,NaN,NaN


### Prepare the data to be inserted ###

In [4]:
# Generate random data
prices = random.sample(range(1000, 10000), 200)
items_in_stock = random.sample(range(100, 1000), 200)
random_ids = random.sample(range(100, 500), 200)
product_ids = [f"P{num}" for num in random_ids]

products_data = pd.DataFrame({"ProductID": product_ids, "Price": prices, "ItemsInStock": items_in_stock},
                             index=range(1, 201))

products_data

,ProductID,Price,ItemsInStock
1,P292,5304,830
2,P171,7486,667
3,P122,8887,265
4,P494,9313,501
5,P490,3648,997
...,...,...,...
196,P131,9936,271
197,P246,1741,146
198,P156,8131,288
199,P335,9814,796


### MySQL Server ###

In [5]:
load_dotenv()
ENV_MYSQL_USER = os.getenv('MYSQL_USER')
ENV_MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
ENV_MYSQL_HOST = os.getenv('MYSQL_HOST')
ENV_MYSQL_DATABASE = os.getenv('MYSQL_DATABASE')

In [6]:
import mysql.connector as connector

# Connect to the database
connection = connector.connect(user=ENV_MYSQL_USER,
                            password=ENV_MYSQL_PASSWORD,
                            host=ENV_MYSQL_HOST,
                            database=ENV_MYSQL_DATABASE)

In [7]:
# Create a link "cursor" between MySQL and Python
cursor = connection.cursor()

In [8]:
cursor.execute("""DELETE FROM Products;""")

In [9]:
# For evaluating INSERT performance
for i in range(200):
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)

    start_time = time.time()
    cursor.execute(add_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 0] = total_time_in_msec

    cursor.execute("""DELETE FROM Products;""")

    time.sleep(0.5)

In [10]:
# For evaluating UPDATE performance
for i in range(200): # Fill the database with data
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)
    cursor.execute(add_record)
    connection.commit()

for i in range(200): # Update randomly
    new_items_in_stock = 50
    product_id = products_data["ProductID"][i+1]
    update_record = """UPDATE Products SET ItemsInStock = {} WHERE ProductID = '{}';""".format(new_items_in_stock, product_id)

    start_time = time.time()
    cursor.execute(update_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 1] = total_time_in_msec
    time.sleep(0.5)

In [11]:
servers_performance[["MySQL/INSERT", "MySQL/UPDATE"]]

,MySQL/INSERT,MySQL/UPDATE
StatementID,,
1,12.372732,17.015457
2,2.008915,1.074076
3,0.000000,0.000000
4,1.008511,1.942635
5,1.086712,0.517130
...,...,...
196,2.008915,1.067400
197,1.056433,1.571655
198,2.020121,1.011610


In [12]:
cursor.close()
connection.close()

### MSSQL ###

In [13]:
load_dotenv()
ENV_MSSQL_SERVER = os.getenv('MSSQL_SERVER')

ENV_MSSQL_DATABASE = os.getenv('MSSQL_DATABASE')

In [14]:
import pyodbc

# Connect to the database using Windows authentication
connection = pyodbc.connect('DRIVER={SQL Server};SERVER='+ENV_MSSQL_SERVER+';DATABASE='+ENV_MYSQL_DATABASE+';Trusted_Connection=yes;')

In [15]:
# Create a link "cursor" between MSSQL and Python
cursor = connection.cursor()

In [16]:
cursor.execute("""DELETE FROM Products;""")

In [17]:
# For evaluating INSERT performance
for i in range(200):
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)

    start_time = time.time()
    cursor.execute(add_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 2] = total_time_in_msec

    cursor.execute("""DELETE FROM Products;""")

    time.sleep(0.5)


In [18]:
# For evaluating UPDATE performance
for i in range(200): # Fill the database with data
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)
    cursor.execute(add_record)
    connection.commit()

for i in range(200): # Update randomly
    new_items_in_stock = 50
    product_id = products_data["ProductID"][i+1]
    update_record = """UPDATE Products SET ItemsInStock = {} WHERE ProductID = '{}';""".format(new_items_in_stock, product_id)

    start_time = time.time()
    cursor.execute(update_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 3] = total_time_in_msec
    time.sleep(0.5)

In [19]:
servers_performance[["MSSQL/INSERT", "MSSQL/UPDATE"]]

,MSSQL/INSERT,MSSQL/UPDATE
StatementID,,
1,29.235601,70.813656
2,0.871658,5.514383
3,0.000000,4.029989
4,0.000000,1.526594
5,1.347780,1.006603
...,...,...
196,2.024889,1.589060
197,2.103567,1.073360
198,1.839399,1.050711


In [20]:
cursor.close()
connection.close()

### PostgreSQL ###

In [21]:
load_dotenv()
ENV_POSTGRES_USER = os.getenv('POSTGRES_USER')
ENV_POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
ENV_POSTGRES_HOST = os.getenv('POSTGRES_HOST')
ENV_POSTGRES_DATABASE = os.getenv('POSTGRES_DATABASE')
ENV_POSTGRES_PORT = os.getenv('POSTGRES_PORT')

In [22]:
import psycopg2

# Connect to the database
connection = psycopg2.connect(host=ENV_POSTGRES_HOST,
                                port=ENV_POSTGRES_PORT,
                                database=ENV_POSTGRES_DATABASE,
                                user=ENV_POSTGRES_USER,
                                password=ENV_POSTGRES_PASSWORD)

In [23]:
# Create a link "cursor" between PostgreSQL and Python
cursor = connection.cursor()

In [24]:
cursor.execute("""DELETE FROM Products;""")

In [25]:
# For evaluating INSERT performance
for i in range(200):
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)

    start_time = time.time()
    cursor.execute(add_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 4] = total_time_in_msec

    cursor.execute("""DELETE FROM Products;""")

    time.sleep(0.5)


In [26]:
# For evaluating UPDATE performance
for i in range(200): # Fill the database with data
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)
    cursor.execute(add_record)
    connection.commit()
    
for i in range(200): # Update randomly
    new_items_in_stock = 50
    product_id = products_data["ProductID"][i+1]
    update_record = """UPDATE Products SET ItemsInStock = {} WHERE ProductID = '{}';""".format(new_items_in_stock, product_id)

    start_time = time.time()
    cursor.execute(update_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 5] = total_time_in_msec
    time.sleep(0.5)

In [27]:
servers_performance[["PostgreSQL/INSERT", "PostgreSQL/UPDATE"]]

,PostgreSQL/INSERT,PostgreSQL/UPDATE
StatementID,,
1,0.996113,1.031160
2,0.000000,0.000000
3,0.000000,0.000000
4,1.010418,1.096010
5,0.000000,0.000000
...,...,...
196,1.053810,1.366138
197,0.000000,1.024485
198,1.011848,1.077890


In [28]:
cursor.close()
connection.close()

### Statistics ###

In [29]:
servers_performance

,MySQL/INSERT,MySQL/UPDATE,MSSQL/INSERT,MSSQL/UPDATE,PostgreSQL/INSERT,PostgreSQL/UPDATE
StatementID,,,,,,
1,12.372732,17.015457,29.235601,70.813656,0.996113,1.031160
2,2.008915,1.074076,0.871658,5.514383,0.000000,0.000000
3,0.000000,0.000000,0.000000,4.029989,0.000000,0.000000
4,1.008511,1.942635,0.000000,1.526594,1.010418,1.096010
5,1.086712,0.517130,1.347780,1.006603,0.000000,0.000000
...,...,...,...,...,...,...
196,2.008915,1.067400,2.024889,1.589060,1.053810,1.366138
197,1.056433,1.571655,2.103567,1.073360,0.000000,1.024485
198,2.020121,1.011610,1.839399,1.050711,1.011848,1.077890


In [30]:
servers_performance.to_csv('serverse-performance.csv')

In [31]:
servers_performance.describe()

,MySQL/INSERT,MySQL/UPDATE,MSSQL/INSERT,MSSQL/UPDATE,PostgreSQL/INSERT,PostgreSQL/UPDATE
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,1.785759,2.869992,2.338682,1.355826,0.653985,0.859050
std,8.148289,12.790483,4.354627,5.000003,0.527105,0.491343
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.514925,1.011431,1.010537,0.528038,0.000000,0.558197
50%,1.039982,1.063704,1.935124,1.052737,1.006126,1.051188
75%,2.009213,1.513839,2.050638,1.428306,1.026392,1.068175
max,114.890575,169.790506,32.122612,70.813656,2.349138,4.007816
